# Post-process vasca_pipe results
Make catalog file of selected sources only, associate and get store additional info for listed sources

In [1]:
region_name = "ALL_10-800" #"CAINGSGII_10-800"  #"TDS" #"CAINGSGII_10-800"  #"TDS" # "MDIS_10-800" # "TDS" # _ELAISN1
region_fname = "./vasca_pipeline/"+region_name+"/region_"+region_name+".fits"
region_cat_fname = region_fname.replace(".fits","_cat.fits")
do_cat = False
do_cross_match = False
do_src_prep = False
do_variable_addition = True

srcs_ids = []
srcs_ids = [55034 ,61491 ,64360 ,73354 ,76243 ,92867 ,104078 ,106745 ,226975 ,267514 ,588567 ,599887 ,]  # WDs ALL

In [2]:
%matplotlib widget
import ipywidgets as widgets
from IPython.display import display
import matplotlib.pyplot as plt
import numpy as np

from astropy.coordinates import SkyCoord
from astropy.table import unique, Table, hstack, join, setdiff
from astropy import units as uu
from astroquery.simbad import Simbad
from astropy.visualization import quantity_support
from astropy.modeling.models import BlackBody
from astropy import constants as cc
from matplotlib.ticker import ScalarFormatter
from astroquery.sdss import SDSS

from vasca.region import Region
from vasca.field import BaseField
import vasca.visualization as vvis
from vasca.tables import TableCollection, dd_vasca_columns
from vasca.utils import otype2ogroup,dd_ogrp2otypes, dd_ogrp2col, dd_filter2wavelength, mag2flux, flux2mag, query_vizier_sed, add_ogrp

from loguru import logger
logger.enable("vasca")

# Do catalog file

if do_cat:
    rg = Region()
    rg.load_from_fits(region_fname)
    rc = rg.get_region_catalog()
else:
    rc = Region()
    rc.load_from_fits(region_cat_fname)

#Cross match to SIMBAD and GAIA
if do_cross_match:
    rc.cross_match_cds(query_table="simbad")
    rc.cross_match_cds()

#Add variables
if do_variable_addition:
    rc.add_column("tt_gaiadr3", "Gmag_abs")
    sel_plx = rc.tt_gaiadr3["RPlx"]>3
    rc.tt_gaiadr3["Gmag_abs"][sel_plx]=rc.tt_gaiadr3["Gmag"][sel_plx] + 5.0 * np.log10(rc.tt_gaiadr3["Plx"][sel_plx]) - 10.0

    # Add object groups
    add_ogrp(rc,"tt_simbad")
    add_ogrp(rc,"tt_otypes")
    add_ogrp(rc,"tt_gaiadr3",provenance="GAIA")

#Write out all changes
rc.write_to_fits(region_cat_fname, overwrite=True)

#Prepare sources, including Vizier SED.
if do_src_prep:
    for src_id in srcs_ids:
        tc_src = rc.get_src_from_id(src_id, load_from_file = False)
print("Done")

2023-10-12 10:36:59.360 | DEBUG    | vasca.tables:load_from_fits:328 - Loading file with name './vasca_pipeline/ALL_10-800/region_ALL_10-800_cat.fits'
2023-10-12 10:36:59.372 | DEBUG    | vasca.tables:load_from_fits:339 - Loading table 'tt_fields'
2023-10-12 10:36:59.399 | DEBUG    | vasca.tables:load_from_fits:339 - Loading table 'tt_filters'
2023-10-12 10:36:59.412 | DEBUG    | vasca.tables:load_from_fits:339 - Loading table 'tt_visits'
2023-10-12 10:36:59.426 | DEBUG    | vasca.tables:load_from_fits:339 - Loading table 'tt_detections'
2023-10-12 10:36:59.474 | DEBUG    | vasca.tables:load_from_fits:339 - Loading table 'tt_coadd_detections'
2023-10-12 10:36:59.579 | DEBUG    | vasca.tables:load_from_fits:339 - Loading table 'tt_sources'
2023-10-12 10:36:59.602 | DEBUG    | vasca.tables:load_from_fits:339 - Loading table 'tt_coadd_sources'
2023-10-12 10:36:59.622 | DEBUG    | vasca.tables:load_from_fits:339 - Loading table 'tt_src_id_map'


KeyError: 'tt_gaiadr3'